In [3]:
import time
import numpy as np

from sklearn.datasets import fetch_openml #download Minist
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [28]:
class MyKnn:

    def loadData(self, test_size=0.2, random_state=100):
        mnist = fetch_openml('mnist_784')
        X = mnist.data/255.0
        y = mnist.target.astype(int)

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
        return np.array(X_train), np.array(y_train), np.array(X_test), np.array(y_test)
        
    def calculate_dist(self, x1,x2,mode='L2') -> np.ndarray:
        '''
        计算两个样本点之间的距离
        params:
            x1: vector 1
            x2: vector 2
            mode: L1 norm or L2 norm
        '''
        if mode=="L2":
            return np.sqrt(np.sum(np.square(x1-x2)))
        else:
            return np.sum(np.abs(x1-x2))
    
    def fit(self, X_train, y_train, x, k=10):
        '''
        用训练集预测新样本x的分类
        '''
        X_train = np.array(X_train)
        m = X_train.shape[0]
        dist_array = np.zeros((m))
        # 1. 计算新样本点与训练集所有点的距离
        for i in range(m):
            #计算样本点与训练集一点的距离
            curDist = self.calculate_dist(x, X_train[i])
            dist_array[i] = curDist
        
        # 2. 对距离排序，选距离最小的k个点，获得他们的index
        top_k_idx = np.argsort(dist_array)[:k]
        top_k_y = np.array([y_train[i] for i in top_k_idx])

        # 3. 选数量最多的作为预测类
        y_pred = np.bincount(top_k_y).argmax()

        return y_pred
    
    def predict(self, X_train, y_train, X_test, y_test, k = 10) -> np.ndarray:
        m = X_test.shape[0]
        y_pred_array = np.zeros((m))
        for i in range(m):
            y_pred_array[i] = self.fit(X_train, y_train, X_test[i], k=k)
        return y_pred_array

In [22]:
train_data, train_label, test_data, test_label = knn.loadData()

/Users/xingwencui/miniconda3/envs/fit5221/lib/python3.9/site-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [29]:
knn = MyKnn()
knn.calculate_dist(train_data[1],test_data[1])

9.663598924588975

In [33]:
y_pred = knn.predict(train_data,train_label,test_data[:100],test_label[:100])

In [34]:
accuracy_score(y_pred, test_label[:100])

0.97